# HW1 - part 4

In [10]:
import sys, random,binascii,AES,jupyter_utils,random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Markdown, display,HTML
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150


This section will focus on the Diffusion principle and examine the extent to which a change in input or internal states affects the end result. The runs in the strain part will be performed on a single input block (16 characters)

In [6]:
plaintext = "From the Pope to"
print("Total Length:",len(plaintext))
display(Markdown(f'\n"**_{plaintext}_**"'))


Total Length: 16



"**_From the Pope to_**"

And we will try to encrypt the message, using an encryption key **"R is a Short key"**
 

In [7]:
key = "R is a Short key"
key = key.encode().hex()
print(f"'R is a Short key':\n 0x{key}")
aes_lib = AES.AES(int(key,base=16))

'R is a Short key':
 0x5220697320612053686f7274206b6579


## Question 4:
1. Run the AES algorithm, and save the intermediate results after each round (including the final result)
2. Randomly select a bit in the login message and change it from 0 -> 1 or 1-> 0 accordingly Repeat (1) and calculate how many bits changed their value as a result of the change of the single bit in each round, and the encrypted message (relative to the same situation when no errors were injected)
4. Repeat the experiment 5 times when each time you change one bit in relation to the message in (1) at the end of the runs, present a table

In [83]:
def run_aes_stages(plaintext):
    spec_enc_text = [""]*10
    for i in range(0, len(plaintext), 16):  
        ciper = aes_lib.encrypt_by_stage(int(plaintext[i:i+16].encode().hex(),base=16))
        for j,c in enumerate(ciper):
            spec_enc_text[j] += ('{:x}'.format(c)).zfill(32)
    return spec_enc_text
spec_enc_text = run_aes_stages(plaintext)
display(spec_enc_text)

['cb95e6a2a56a602164f8bdc338622f85',
 'cbf42cb3f27bbb6266e1e7abbc87199e',
 '5953b07e2a304dc27be50e6355176310',
 '203d5627e023761e3d86efd4ace38e38',
 'b874b055f6f6fc733e8c7952dbf7ea9c',
 '1dba47880ed16f3c72d966c2526347cc',
 '737bf4f4317604824733b3b13695b2cf',
 'd0d5082d7e23c65c2e2cd9304833f9f6',
 '54d92855bb1e7c8075c0eb813950e85d',
 '793ec12a27911757630e1fd3c4affd06']

For stage II and on, we wants' to create a convertion function

In [78]:
def tobits(s):
    result = []
    for c in s:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        result.extend([int(b) for b in bits])
    return result

def frombits(bits):
    chars = []
    for b in range(len(bits) // 8 ):
        byte = bits[b*8:(b+1)*8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars)

def count_flips_to_convert(a, b):

    diff = a ^ b

    # count number of ones in diff
    count = 0
    while diff:
        diff &= (diff - 1)
        count += 1
    return count

In [82]:
def flip_plaintext(plaintext):
    bitflip_p = tobits(plaintext)
    place = random.choice(range(len(bitflip_p)))
    bitflip_p[place] = int(1 - bitflip_p[place])
    bitflip_plaintext = frombits(bitflip_p)
    return bitflip_plaintext,place
bitflip_plaintext,place = flip_plaintext(plaintext)
print("The bit which was chosen to be replaced: ", place)
print("Result:",bitflip_plaintext)

The bit which was chosen to be replaced:  35
Result: From0the Pope to


In [85]:
bitflip_spec_enc_text = run_aes_stages(bitflip_plaintext)
display(bitflip_spec_enc_text)

['cb95e6a2ecc3c9c164f8bdc338622f85',
 'bfac009f79b37321e36473bab1900393',
 '6f26bfa59b3e4de0e34886ce95df8b80',
 'cf044fd5ac7b9faac48423964571d2a3',
 '3f5a74f658909ed16e49e33f615f6e1b',
 '9f64bac419d1f1641e8fce1e44cecf4b',
 'cd2ff326f7231ac44fb5ba889cb7f5f2',
 '0c6808fa98801ee145d71073e57ca2fc',
 '7872a5371d5eef691790aaccb38d3456',
 'e51484d7694b94310e002de7bbda3241']

In [86]:
def calulate_diffrence(textlist,flip_textlist):
    diff = []
    for i in range(len(flip_textlist)):
        p =  int(textlist[i],base=16)
        c =  int(flip_textlist[i],base=16)
        diff.append(count_flips_to_convert(p,c))
    return diff
diff = calulate_diffrence(bitflip_spec_enc_text,spec_enc_text)
display(diff)

[14, 50, 54, 64, 59, 60, 56, 72, 57, 69]

In [102]:
table_result = np.zeros((6,len(bitflip_spec_enc_text)))

for i in range(5):
    bfp,place = flip_plaintext(plaintext)
    bfp_stage = run_aes_stages(bfp)
    table_result[i] = calulate_diffrence(bfp_stage,spec_enc_text)


In [110]:
table_result[5] = np.mean(table_result, axis = 0)
display(pd.DataFrame(table_result, columns=[f"Round {col+1}" for col in range(len(bitflip_spec_enc_text))]))

,Round 1,Round 2,Round 3,Round 4,Round 5,Round 6,Round 7,Round 8,Round 9,Round 10
0,15.000000,65.000000,63.000000,62.000000,62.000000,52.000000,69.000000,63.000000,56.000000,72.000000
1,17.000000,77.000000,59.000000,70.000000,66.000000,80.000000,68.000000,59.000000,57.000000,64.000000
2,55.000000,57.000000,71.000000,59.000000,58.000000,57.000000,67.000000,62.000000,63.000000,79.000000
3,20.000000,77.000000,72.000000,60.000000,58.000000,66.000000,62.000000,61.000000,63.000000,65.000000
4,15.000000,50.000000,72.000000,62.000000,68.000000,64.000000,68.000000,68.000000,60.000000,57.000000
5,23.722222,63.388889,65.527778,60.861111,60.666667,62.027778,64.944444,60.861111,58.138889,65.527778
